# 02 — Feature Research

**Цель:** найти сильные фичи для выявления мошенничества.
- Проверяем утечки (data leakage)
- Делаем фичи по времени, географии, устройствам, поведенческие агрегаты
- Валидируем на простых моделях


In [ ]:
# Базовые импорты
import json, numpy as np, pandas as pd
import pyarrow.dataset as ds
from collections import defaultdict, Counter

path_tx = "/content/drive/MyDrive/ИТМО/transaction_fraud_data.parquet"
path_fx = "/content/drive/MyDrive/ИТМО/historical-currency-exchange.csv"

# Если работаешь в Colab, не забудь смонтировать Google Drive до запуска:
# from google.colab import drive; drive.mount('/content/drive')


In [ ]:
# Быстрые sanity-checks
ds_tx = ds.dataset(path_tx, format="parquet")
cols = ds_tx.schema.names
print("Колонки:", cols)
print("Пример данных:")
print(ds_tx.head(3).to_pandas())


## Кандидаты фич
1. **Временные:** час/день недели/праздники, частота клиента за окно.
2. **Гео:** дистанция между домашним городом клиента и текущей транзакцией; частота "вне дома".
3. **Девайсы/канал:** количество уникальных устройств/каналов на клиента за N дней.
4. **Деньги:** amount в валюте USD; нормировка на медианный чек клиента.
5. **Поведение:** кол-во уникальных мерчантов/стран за окно; всплески активности.


In [ ]:
# Пример: онлайновые агрегаты по клиенту
usecols = ["customer_id","timestamp","amount","currency","country","city","device","is_fraud"]
dataset = ds.dataset(path_tx, format="parquet")
per_client_last_24h = defaultdict(lambda: [0,0.0, set(), set()]) # cnt, sum, countries, devices

for b in dataset.to_batches(columns=usecols, batch_size=1_000_000):
    df = b.to_pandas()
    df["ts"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.sort_values("ts")
    # тут можно добавить скользящее окно; для примера ограничимся группировками
    g = df.groupby("customer_id").agg(cnt=("amount","size"), amt_sum=("amount","sum"),
                                      n_countries=("country",lambda x: x.nunique()),
                                      n_devices=("device",lambda x: x.nunique()))
    print(g.head())
    break  # для примера выходим после первого батча


## Дальше
- Сохранить конструированные фичи (parquet/csv) в `data/` (локально, в гит не коммитить).
- Быстрая валидация: train/val split по времени, baseline `LogisticRegression`/`XGBoost`.
- Метрики: ROC-AUC, PR-AUC; важности фич; stability по времени.
